# Analisis de prestaciones por diagnostico

- En este apartado sequiere ver la cantidad de prestaciones que se realizan por cada diagnostico principal.

- El resultado que se quiere obtener es parecido al siguiente:

| Año  | Diagnostico | Resumen Procedimientos     |
| ---- | ----------- | -------------------------- |
| 2019 | "I25.1"     | {"45.23": 20, "73.59": 10} |
| 2020 | "I25.1"     | {"45.23": 7, "73.59": 3}   |


In [53]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_DEL_TORAX = 112103


In [55]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";")

In [56]:
procedimientos_long = pd.melt(
    df_procesada,
    id_vars=["COD_HOSPITAL", "ANIO_EGRESO", "DIAGNOSTICO1"],
    value_vars=[f"PROCEDIMIENTO{i}" for i in range(1, 31)],
)

In [57]:
conteo_procedimientos = procedimientos_long.groupby(["ANIO_EGRESO", "COD_HOSPITAL"])[
    "value"
].value_counts()

In [58]:
conteo_procedimientos = conteo_procedimientos.reset_index(name="conteo")

In [59]:
conteo_procedimientos = conteo_procedimientos.sort_values(
    ["ANIO_EGRESO", "value", "conteo"], ascending=False
)


In [76]:
conteo_procedimientos["ranking"] = (
    conteo_procedimientos.groupby(["ANIO_EGRESO", "value"]).cumcount() + 1
)

conteo_procedimientos["diferencia_proximo_hospital"] = conteo_procedimientos["conteo"].diff(
    periods=-1
)

conteo_procedimientos["diferencia_proximo_hospital"] = np.where(
    conteo_procedimientos["diferencia_proximo_hospital"] < 0,
    -1,
    conteo_procedimientos["diferencia_proximo_hospital"],
)

In [77]:
mejores_procedimientos_torax = conteo_procedimientos.query(
    "COD_HOSPITAL == @HOSPITAL_DEL_TORAX and ranking == 1"
)

In [79]:
mejores_procedimientos_torax.query("diferencia_proximo_hospital == -1")

,ANIO_EGRESO,COD_HOSPITAL,value,conteo,ranking,diferencia_proximo_hospital
177480,2021.0,112103,87.93,1,1,-1.0
177425,2021.0,112103,38.22,1,1,-1.0
177399,2021.0,112103,33.25,1,1,-1.0
177380,2021.0,112103,5.11,1,1,-1.0
107904,2020.0,112103,40.69,1,1,-1.0
107704,2020.0,112103,37.6,5,1,-1.0
107880,2020.0,112103,37.37,1,1,-1.0
107775,2020.0,112103,33.48,2,1,-1.0
